In [1]:
import os
import time
import Turbine
import mat_properties as prop
import numpy as n
import pandas as pd

os.environ["RPPREFIX"] = r"C:/Program Files (x86)/REFPROP"

RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")

water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)


# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
# таблица задаваемых значений (заменится на интерфейс)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)

water_streams['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S,X
DROSVD-TURBVD,508.600000,5.1990,3414.093682,63.40,NaN,NaN
ENDOFVD,195.852578,0.5490,2844.285130,63.40,NaN,NaN
DROSND-TURBND,209.800000,0.4000,2859.669112,14.76,NaN,NaN
SMESHEND,197.800000,0.5490,2848.521603,78.16,NaN,NaN
DOOTB2,131.400000,0.2542,2724.861533,78.16,NaN,NaN
DOOTB1,NaN,0.1525,2652.000000,78.16,NaN,NaN
INCND,NaN,0.1525,2652.000000,78.16,NaN,NaN
INKOND,41.510000,0.0017,2338.024560,78.16,NaN,NaN


In [2]:
calctolerance = 10**-3
calcmethod = "hybr"
KPD_SP = 0.99  # эффективность сетевых подогревателей

Turb = Turbine.turbine(
    water,
    "DROSVD-TURBVD",
    "ENDOFVD",
    "DROSND-TURBND",
    "SMESHEND",
    "DOOTB2",
    "DOOTB1",
    "INCND",
    "INKOND",
    water_streams0,
    water_streams,
)

start_time = time.time()

res = Turb.calculate(diafragma=0.2, maxiterations=50, calctolerance=10**-5)

print("fin:--- %s сек. ---" % round((time.time() - start_time), 1))
res

1.9382942032000192
1.0068000000000006
0.8534579085240119
1.0068000000000006
0.837518512945534
1.0068000000000006
fin:--- 0.1 сек. ---


[0.8462130543785595,
 0.8207056532332418,
 0.7719792559731901,
 0.8432136388335641]

In [3]:
# надо посчитать потерю давления НД (готово)
# отборы неправильно (второй отбор завышен, надопроверить исходные данные функции)
water_streams['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S,X
DROSVD-TURBVD,505.102824,7.830655,3414.093682,63.40,6.754673,100.00000
ENDOFVD,207.638473,0.645430,2865.023235,63.40,6.965945,100.00000
DROSND-TURBND,206.628374,0.707430,2859.669112,14.76,6.913926,100.00000
SMESHEND,207.177169,0.645430,2864.012144,78.16,6.963841,100.00000
DOOTB2,167.467862,0.419493,2790.215862,78.16,6.996022,100.00000
DOOTB1,155.598743,0.362063,2768.419081,78.16,7.011763,100.00000
INCND,148.245130,0.162063,2768.419081,78.16,7.374969,100.00000
INKOND,54.043168,0.015054,2457.836275,78.16,7.576193,94.07386


In [4]:
water_streams0['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S
DROSVD-TURBVD,508.600000,7.8450,3422.525472,63.40,NaN
ENDOFVD,195.852578,0.5490,2844.285130,63.40,NaN
DROSND-TURBND,209.800000,0.6110,2871.426945,14.76,7.003739
SMESHEND,197.800000,0.5490,2848.521603,78.16,7.003601
DOOTB2,131.400000,0.2542,2724.861533,78.16,7.065770
DOOTB1,NaN,0.1525,2652.000000,78.16,7.109000
INCND,NaN,0.1525,2652.000000,78.16,NaN
INKOND,41.510000,0.0080,2341.254319,78.16,NaN


In [5]:
Get_value='P'
Error=(water_streams['DROSVD-TURBVD':'INKOND'][Get_value]-water_streams0['DROSVD-TURBVD':'INKOND'][Get_value])/water_streams0['DROSVD-TURBVD':'INKOND'][Get_value]*100
Error

DROSVD-TURBVD     -0.182854
ENDOFVD           17.564602
DROSND-TURBND     15.782269
SMESHEND          17.564602
DOOTB2            65.024750
DOOTB1           137.418375
INCND              6.270834
INKOND            88.169261
Name: P, dtype: float64

In [6]:
Turb.calculate_power()

{'Ni': 66.55777229754736,
 'Ni1': 34.8110663564488,
 'Ni2': 5.767917454407764,
 'Ni3': 24.2751521189697,
 'Ni4': 1.7036363677210897}